In [1]:
!pip install numpy
!pip install tensorflow
!pip install tsfel
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, Dense, Flatten, Dropout, MaxPooling1D
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tsfel import feature_extraction


In [19]:
# Define dataset folder path
extract_path = "D:/HAR/UCI HAR Dataset"

In [21]:
# List the extracted files and folders
import os
extracted_files = os.listdir(extract_path)
extracted_files

['UCI HAR Dataset', '__MACOSX']

In [23]:
# List contents of the main dataset folder
dataset_main_path = os.path.join(extract_path, "UCI HAR Dataset")
dataset_contents = os.listdir(dataset_main_path)
dataset_contents




['.DS_Store',
 'activity_labels.txt',
 'features.txt',
 'features_info.txt',
 'README.txt',
 'test',
 'train']

In [27]:
# List contents of the 'train' and 'test' folders
train_folder = os.path.join(dataset_main_path, "train")
test_folder = os.path.join(dataset_main_path, "test")

train_contents = os.listdir(train_folder)
test_contents = os.listdir(test_folder)

train_contents, test_contents



(['Inertial Signals', 'subject_train.txt', 'X_train.txt', 'y_train.txt'],
 ['Inertial Signals', 'subject_test.txt', 'X_test.txt', 'y_test.txt'])

In [29]:
# List contents of the 'Inertial Signals' folder inside train and test
train_inertial_path = os.path.join(train_folder, "Inertial Signals")
test_inertial_path = os.path.join(test_folder, "Inertial Signals")

train_inertial_files = os.listdir(train_inertial_path)
test_inertial_files = os.listdir(test_inertial_path)

train_inertial_files, test_inertial_files


(['body_acc_x_train.txt',
  'body_acc_y_train.txt',
  'body_acc_z_train.txt',
  'body_gyro_x_train.txt',
  'body_gyro_y_train.txt',
  'body_gyro_z_train.txt',
  'total_acc_x_train.txt',
  'total_acc_y_train.txt',
  'total_acc_z_train.txt'],
 ['body_acc_x_test.txt',
  'body_acc_y_test.txt',
  'body_acc_z_test.txt',
  'body_gyro_x_test.txt',
  'body_gyro_y_test.txt',
  'body_gyro_z_test.txt',
  'total_acc_x_test.txt',
  'total_acc_y_test.txt',
  'total_acc_z_test.txt'])

In [31]:
# Load a sample inertial sensor file to check its format
import numpy as np

sample_file_path = os.path.join(train_inertial_path, "body_acc_x_train.txt")

# Load data from the text file
sample_data = np.loadtxt(sample_file_path)

# Check shape and a small sample of the data
sample_data.shape, sample_data[:5]


((7352, 128),
 array([[ 1.808515e-04,  1.013856e-02,  9.275574e-03,  5.065897e-03,
          1.081025e-02,  4.045101e-03,  4.757375e-03,  6.213647e-03,
          3.306744e-03,  7.571941e-03,  5.407117e-03,  6.221276e-03,
          6.895028e-03,  4.610260e-03,  7.331333e-03,  5.077667e-03,
          5.762556e-03,  6.692748e-03,  5.443238e-03,  8.240952e-03,
          6.506451e-03,  6.531523e-03,  7.422441e-03,  5.771769e-03,
          6.240187e-03,  4.622982e-03,  4.832962e-03,  5.498746e-03,
          4.341186e-03,  5.098018e-03,  4.268742e-03,  3.045187e-03,
          3.204192e-03,  4.520400e-03,  5.127211e-03,  3.549791e-03,
          4.234233e-03,  4.468670e-03,  3.572702e-03,  5.136278e-03,
          5.379786e-03,  4.408520e-03,  2.244039e-03, -1.183598e-03,
         -6.795342e-04,  2.281389e-03,  3.653728e-03,  2.312639e-03,
         -4.042194e-05,  5.342254e-04,  2.963263e-03,  4.399468e-03,
          2.702339e-03,  3.452310e-04,  2.916911e-03,  4.978556e-03,
          3.446870e-

In [39]:
# Function to load inertial sensor data correctly
def load_inertial_signals(folder, dataset_type="train"):
    signals = []
    signal_filenames = [
        "body_acc_x_", "body_acc_y_", "body_acc_z_",
        "body_gyro_x_", "body_gyro_y_", "body_gyro_z_",
        "total_acc_x_", "total_acc_y_", "total_acc_z_"
    ]

    for signal in signal_filenames:
        file_path = os.path.join(folder, f"{signal}{dataset_type}.txt")  # Corrected filename pattern
        signals.append(np.loadtxt(file_path))

    return np.transpose(np.array(signals), (1, 2, 0))  # Reshape to (samples, time_steps, channels)

# Load training and test inertial data
X_train = load_inertial_signals(train_inertial_path, dataset_type="train")
X_test = load_inertial_signals(test_inertial_path, dataset_type="test")

# Check shape of loaded data
X_train.shape, X_test.shape


((7352, 128, 9), (2947, 128, 9))

In [41]:
# Normalize data to range [-1, 1]
X_train = X_train / np.max(np.abs(X_train))
X_test = X_test / np.max(np.abs(X_test))

# Encode labels as integers (0 to 5) for categorical classification
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_labels)
y_test_encoded = label_encoder.transform(y_test_labels)

# Check encoded labels
np.unique(y_train_encoded, return_counts=True)


(array([0, 1, 2, 3, 4, 5], dtype=int64),
 array([1407, 1286, 1374, 1226,  986, 1073], dtype=int64))

In [133]:
# Define paths
dataset_folder = "D:/HAR/UCI HAR Dataset/UCI HAR Dataset"
train_folder = os.path.join(dataset_folder, "train")
test_folder = os.path.join(dataset_folder, "test")
inertial_signals_folder_train = os.path.join(train_folder, "Inertial Signals")
inertial_signals_folder_test = os.path.join(test_folder, "Inertial Signals")


In [135]:
# Load activity labels
activity_labels = pd.read_csv(os.path.join(dataset_folder, "activity_labels.txt"), 
                              sep=' ', header=None, index_col=0)[1]

In [139]:
# Load raw inertial data
def load_inertial_data(folder, prefix, dataset_type):
    signals = []
    for axis in ['x', 'y', 'z']:
        filepath = os.path.join(folder, f"{prefix}_{axis}_{dataset_type}.txt")
        if os.path.exists(filepath):
            signals.append(np.loadtxt(filepath))
        else:
            raise FileNotFoundError(f"File {filepath} not found.")
    return np.stack(signals, axis=-1)  # Shape: (samples, timesteps, features)



In [141]:
X_train_raw = load_inertial_data(inertial_signals_folder_train, "body_acc", "train")
X_test_raw = load_inertial_data(inertial_signals_folder_test, "body_acc", "test")


In [143]:
y_train = np.loadtxt(os.path.join(train_folder, "y_train.txt"))
y_test = np.loadtxt(os.path.join(test_folder, "y_test.txt"))


In [145]:
# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train - 1, num_classes=6)
y_test = tf.keras.utils.to_categorical(y_test - 1, num_classes=6)


In [147]:
# Define LSTM model
lstm_model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train_raw.shape[1], X_train_raw.shape[2])),
    LSTM(32),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_raw, y_train, epochs=10, batch_size=32, validation_data=(X_test_raw, y_test))


C:\Users\mdash\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 30s 114ms/step - accuracy: 0.2433 - loss: 1.6806 - val_accuracy: 0.3712 - val_loss: 1.1691
Epoch 2/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 26s 111ms/step - accuracy: 0.3444 - loss: 1.2193 - val_accuracy: 0.4109 - val_loss: 1.0886
Epoch 3/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 25s 110ms/step - accuracy: 0.4297 - loss: 1.0461 - val_accuracy: 0.4751 - val_loss: 0.9704
Epoch 4/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 14s 61ms/step - accuracy: 0.4632 - loss: 0.9742 - val_accuracy: 0.4595 - val_loss: 0.9516
Epoch 5/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 14s 61ms/step - accuracy: 0.4592 - loss: 1.0214 - val_accuracy: 0.4550 - val_loss: 1.0640
Epoch 6/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 17s 75ms/step - accuracy: 0.4987 - loss: 0.9053 - val_accuracy: 0.5331 - val_loss: 0.8909
Epoch 7/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 18s 76ms/step - accuracy: 0.5624 - loss: 0.8449 - val_accuracy: 0.5419 - val_loss: 0.8910
Epoch 8/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 15s 65ms/step - accuracy: 0.5718 - loss: 0.8051

In [149]:
# Define 1D CNN model
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_raw.shape[1], X_train_raw.shape[2])),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_raw, y_train, epochs=10, batch_size=32, validation_data=(X_test_raw, y_test))


Epoch 1/10


C:\Users\mdash\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


230/230 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.3375 - loss: 1.4444 - val_accuracy: 0.5219 - val_loss: 0.8870
Epoch 2/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4829 - loss: 0.9757 - val_accuracy: 0.6111 - val_loss: 0.7657
Epoch 3/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5483 - loss: 0.8529 - val_accuracy: 0.6016 - val_loss: 0.7337
Epoch 4/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5434 - loss: 0.8084 - val_accuracy: 0.5986 - val_loss: 0.7260
Epoch 5/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5768 - loss: 0.7687 - val_accuracy: 0.5881 - val_loss: 0.7626
Epoch 6/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5987 - loss: 0.7300 - val_accuracy: 0.6566 - val_loss: 0.7047
Epoch 7/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5979 - loss: 0.7168 - val_accuracy: 0.6077 - val_loss: 0.7139
Epoch 8/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6025 - loss: 0.7135 - val_accuracy: 0.6410 - val_

In [177]:
# Train 1D CNN model after LSTM
combined_model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train_raw.shape[1], X_train_raw.shape[2])),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
combined_model.fit(X_train_raw, y_train, epochs=10, batch_size=32, validation_data=(X_test_raw, y_test))


C:\Users\mdash\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - accuracy: 0.3498 - loss: 1.4314 - val_accuracy: 0.4130 - val_loss: 1.1868
Epoch 2/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.5040 - loss: 0.9640 - val_accuracy: 0.5626 - val_loss: 0.7793
Epoch 3/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.5411 - loss: 0.8683 - val_accuracy: 0.5127 - val_loss: 0.8275
Epoch 4/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.5606 - loss: 0.8180 - val_accuracy: 0.5874 - val_loss: 0.7812
Epoch 5/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.5828 - loss: 0.7644 - val_accuracy: 0.5925 - val_loss: 0.7695
Epoch 6/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.5783 - loss: 0.7729 - val_accuracy: 0.5894 - val_loss: 0.7564
Epoch 7/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - accuracy: 0.5647 - loss: 0.7551 - val_accuracy: 0.6023 - val_loss: 0.8406
Epoch 8/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - accuracy: 0.5753 - loss: 0.7509 - val_ac

In [203]:
import scipy.stats

# Ensure compatibility
if not hasattr(scipy.stats, "median_absolute_deviation"):
    scipy.stats.median_absolute_deviation = scipy.stats.median_abs_deviation


In [ ]:
import scipy
min_length = 10  # Adjust as needed

def extract_features(samples, cfg):
    feature_list = []
    for sample in samples:
        if sample.shape[0] >= min_length:
            features = tsfel.time_series_features_extractor(cfg, sample[:, 0], fs=50)
            feature_list.append(features.values)
        else:
            print("Skipping short signal:", sample.shape)
            feature_list.append(np.zeros((1, len(cfg))))  # Placeholder for short signals
    return np.vstack(feature_list)

X_train_features = extract_features(X_train_raw, cfg)
X_test_features = extract_features(X_test_raw, cfg)

*** Feature extraction started ***
16


*** Feature extraction started ***
16


*** Feature extraction started ***
16


*** Feature extraction started ***
16


*** Feature extraction started ***
16


...

*** Feature extraction started ***
16


*** Feature extraction started ***
16


*** Feature extraction started ***
16


*** Feature extraction started ***
16


*** Feature extraction started ***
16


In [223]:
X_train_features = np.nan_to_num(X_train_features)
X_test_features = np.nan_to_num(X_test_features)


In [225]:
if X_train_features.ndim == 1:
    X_train_features = X_train_features.reshape(len(X_train_features), -1)
if X_test_features.ndim == 1:
    X_test_features = X_test_features.reshape(len(X_test_features), -1)


In [229]:
# Train Random Forest
rf = RandomForestClassifier()
rf.fit(X_train_features, np.argmax(y_train, axis=1))
y_pred_rf = rf.predict(X_test_features)
accuracy_rf = accuracy_score(np.argmax(y_test, axis=1), y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf:.4f}")

Random Forest Accuracy: 0.5928


In [231]:
# Train SVM
svm = SVC()
svm.fit(X_train_features, np.argmax(y_train, axis=1))
y_pred_svm = svm.predict(X_test_features)
accuracy_svm = accuracy_score(np.argmax(y_test, axis=1), y_pred_svm)
print(f"SVM Accuracy: {accuracy_svm:.4f}")


SVM Accuracy: 0.5358


In [233]:
# Train Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_features, np.argmax(y_train, axis=1))
y_pred_log_reg = log_reg.predict(X_test_features)
accuracy_log_reg = accuracy_score(np.argmax(y_test, axis=1), y_pred_log_reg)
print(f"Logistic Regression Accuracy: {accuracy_log_reg:.4f}")


Logistic Regression Accuracy: 0.5728


C:\Users\mdash\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
